In [1]:
from nltk.tokenize.casual import casual_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import pandas as pd
import string
import os
base = "../../data/clean/"
pd.set_option('display.max_rows', 150)

In [2]:
tweetdf = pd.read_csv(base+'loctrisma2016.csv')
tweetdf = tweetdf.sort_values(by='tweet_id')
tweetdf.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,tweet_id,created_at,from_user_id,source,text,to_user_id,to_user_name,...,location_name,country,location_full_name,location_type,country_code,collected_at,lga,state,lat,lng
375922,375922,412183,246385,682712842600972291,2016-01-01 00:00:00.000000,106082182,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@legallyfm @XFilesNews @thexfiles @mynewsaga &...,39023478.0,legallyfm,...,Glen Innes,Australia,"Glen Innes, New South Wales",city,AU,2013-01-01 00:00:01,Glen Innes Severn,NSW,-29.751838,151.736189
33932,33932,34218,503413,682712878915260418,2016-01-01 00:00:09.000000,269023896,"<a href=""http://sandaysoft.com/"" rel=""nofollow...",Temp: 33.1°C Wind: 5kph Pressure: 995.9hpa Fal...,NaN,NaN,...,Baradine,Australia,"Baradine, New South Wales",city,AU,2013-01-01 00:00:01,Warrumbungle Shire,NSW,-30.950000,149.066667
266436,266436,291148,502347,682712884565049344,2016-01-01 00:00:10.000000,379712969,"<a href=""http://sandaysoft.com/"" rel=""nofollow...","Wind 2.7 kts NE. Barometer 1017.9 mb, Falling ...",NaN,NaN,...,Raymond Terrace,Australia,"Raymond Terrace, New South Wales",city,AU,2013-01-01 00:00:01,Port Stephens,NSW,-32.754408,151.767512
132916,132916,145044,79326,682712982909005824,2016-01-01 00:00:33.000000,109140867,"<a href=""http://instagram.com"" rel=""nofollow"">...","Best coffee, food and swevice on the Hume and ...",NaN,NaN,...,Berremangra,Australia,"Berremangra, New South Wales",neighborhood,AU,2013-01-01 00:00:01,Hilltops,NSW,-34.830222,148.516892
565350,565350,623155,1323531,682713033836085248,2016-01-01 00:00:46.000000,178662965,"<a href=""http://twitter.com/download/android"" ...",Best start to 2016 - a morning at my favourite...,NaN,NaN,...,Dudley,Australia,"Dudley, Newcastle",neighborhood,AU,2013-01-01 00:00:01,Lake Macquarie,NSW,-32.990000,151.718000


In [3]:
by_tweeter = tweetdf.groupby(['lga','from_user_id'], as_index=False).agg({'text':lambda x:' '.join(x)})
by_tweeter

,lga,from_user_id,text
0,Albury,766581,Who actually thinks it is ok to bring a McCafe...
1,Albury,2405601,"Road trip playlist \nHappy Friday, my funky bu..."
2,Albury,5372792,“@heyjudeonline: Amazing day working with the ...
3,Albury,5641092,What exactly is the point of a “stay signed in...
4,Albury,5918802,"I'm at Commercial Club in Albury, NSW https://..."
5,Albury,6221342,I just finished running 11.17 km in 1h:11m:48s...
6,Albury,6434942,@Elusive_Sausage she is so beautiful!!! @kardi...
7,Albury,6518622,@PolarBearFarm they're all dodging taxes and t...
8,Albury,6916182,hello New South Wales how are you @phillfarrug...
9,Albury,12266342,"I'm at Early Bird Cafe in Albury, NSW https://..."


In [4]:
import string
import emoji
from unidecode import unidecode
from tokenizer import tokenizer
# Remove handles and urls but not hash tags.
# Handles might be useful for an is-reply feature but not for analysis of the text
smc_tokenizer = tokenizer.TweetTokenizer(preserve_handles=False, preserve_hashes=True, preserve_len=False,
                                         preserve_case=False, preserve_url=False)


In [5]:
# First, remove all emojis, then decode all unicode, then put emojis at the end
docs = []
for doc in by_tweeter['text']:
    emojis = ' '.join([c for c in doc if c in emoji.UNICODE_EMOJI])
    stripped = ''.join([c for c in doc if c not in emoji.UNICODE_EMOJI])
    decoded = unidecode(stripped)
    docs.append(' '.join([decoded, emojis]))
len(docs)

58274

In [6]:
# TweetTokenizer weirdly keeps punctuation separated by space as one token:
puncts = set(string.punctuation + ' ')
docs = [smc_tokenizer.tokenize(doc) for doc in docs]
cleaned = []
for doc in docs:
    clean = [token for token in doc if not(len(token) == 1 and token not in emoji.UNICODE_EMOJI)
                and not(all(c in puncts for c in token))]
    cleaned.append(' '.join(clean))

In [7]:
cleaned[0:3]

['who actually thinks it is ok to bring mccafe drink into normal cafe who actually thinks it is ok to bring mccafe drink into normal cafe',
 'road trip playlist happy friday my funky bunch #nowplaying #playitloud',
 'amazing day working with the film crew to create online video resources']

In [8]:
# Add cleaned tokens as an attribute to by_tweeter:
import pandas as pd
cs = pd.Series(cleaned)
by_tweeter['cleaned'] = cs
by_tweeter

,lga,from_user_id,text,cleaned
0,Albury,766581,Who actually thinks it is ok to bring a McCafe...,who actually thinks it is ok to bring mccafe d...
1,Albury,2405601,"Road trip playlist \nHappy Friday, my funky bu...",road trip playlist happy friday my funky bunch...
2,Albury,5372792,“@heyjudeonline: Amazing day working with the ...,amazing day working with the film crew to crea...
3,Albury,5641092,What exactly is the point of a “stay signed in...,what exactly is the point of stay signed in bo...
4,Albury,5918802,"I'm at Commercial Club in Albury, NSW https://...",i'm at commercial club in albury nsw
5,Albury,6221342,I just finished running 11.17 km in 1h:11m:48s...,just finished running 11.17 km in 1h 11m 48s w...
6,Albury,6434942,@Elusive_Sausage she is so beautiful!!! @kardi...,she is so beautiful happy chocolate day may th...
7,Albury,6518622,@PolarBearFarm they're all dodging taxes and t...,they're all dodging taxes and thus have lower ...
8,Albury,6916182,hello New South Wales how are you @phillfarrug...,hello new south wales how are you oh see that ...
9,Albury,12266342,"I'm at Early Bird Cafe in Albury, NSW https://...",i'm at early bird cafe in albury nsw


In [9]:
by_tweeter.to_csv('../../data/clean/trisma2016_cleaned_text_by_lga_and_from_user_id.csv')